# Introduction

## 1.1 Background
Jollibee is the largest fast food restaurant chain in the Philippines, while very well known around Southeast Asia and Middle East, there is only one Jollibee franchise in Toronto Canada.

## 1.2 Business Problem
Jollibee Foods Corporation (JFC) looks to expand its footprint in Toronto, Canada. As of July 2020, the only Jollibee franchise is in Scarborough and the company is looking for a similar neighborhood where they can open another franchise.

# Data

## 2.1 Data Sources
The neighbourhoods alongside their respective postal codes and boroughs were scraped from [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Geographical coordinates for each neighbourhood were extracted from [here](https://cocl.us/Geospatial_data). I'll also be using the median household income data that is available [here](https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2270&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=SurveyZone&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian).

I will be utilizing the Foursquare API to obtain/explore which neighborhoods has the highest frequency of fast food chains.

## 2.2 Collection
### 2.2.1 Scraping Toronto, Canada Wikipedia Page

In [1]:
import urllib.request
from bs4 import BeautifulSoup


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

Looking at the Wikipedia page, there's only one table in it and its class attribute is wikitable sortable. Using that information and the find function of the BeautifulSoup package, we find the table xml

In [2]:
postal_code_table = soup.find('table', class_='wikitable sortable')

Knowing how HTML tables are written, I looped through each `tr` of the HTML table and use follow the instructions:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [29]:
PostalCode = []
Borough = []
Neighborhood = []

for row in postal_code_table.findAll('tr')[1:]:
    cells = row.findAll('td')
    borough = cells[1].find(text=True).strip()
    
    if borough != 'Not assigned':
        PostalCode.append(cells[0].find(text=True).strip())
        Borough.append(borough)
        neighborhood = cells[2].find(text=True).strip()
        if neighborhood == 'Not assigned': 
            Neighborhood.append(borough)
        else:
            Neighborhood.append(neighborhood)
            

data = []
for i, N in enumerate(Neighborhood):
    b = Borough[i]
    p = PostalCode[i]
    for n in N.split(','):
        n = n.strip()
        if '-' in n:
            n.replace(' - ', '-')

        d = {
            'Borough': b,
            'PostalCode': p,
            'Neighborhood': n.strip()
        }

        data.append(d)
        
neighborhood_df = pd.DataFrame(data)
neighborhood_df.head()

,Borough,PostalCode,Neighborhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,Regent Park
3,Downtown Toronto,M5A,Harbourfront
4,North York,M6A,Lawrence Manor


### 2.2.2 Load Population, Ethnic, and Income Data

In [11]:
# load total population of each Toronto neighbourhood
pop_path = pd.read_csv('population.csv')
df_pop = pd.DataFrame(pop_path)
df_pop.drop(['Neighborhood Id'], axis=1, inplace=True)

# load population of Southeast Asians in Toronto neighbourhoods
ethnic_path = pd.read_csv('ethnic.csv')
df_ethnic = pd.DataFrame(ethnic_path)
df_ethnic.drop(['Neighborhood Id'], axis=1, inplace=True)
df_ethnic.rename(columns={'   Southeast Asian': 'Southeast Asian'}, inplace=True)

# load household income of each Toronto neighbourhood
income_path = pd.read_csv('income.csv')
df_income = pd.DataFrame(income_path)
df_income.drop(['Neighborhood Id'], axis=1, inplace=True)

print("{}, {}, {}".format(df_ethnic.shape[1], df_pop.shape[1], df_income.shape[1]))

2, 2, 2


,Neighborhood,Neighborhood Id,Southeast Asian
0,West Humber-Clairville,1.0,490.0
1,Mount Olive-Silverstone-Jamestown,2.0,590.0
2,Thistletown-Beaumond Heights,3.0,120.0
3,Rexdale-Kipling,4.0,170.0
4,Elms-Old Rexdale,5.0,370.0


Next, let's merge all three dataframes.

In [13]:
from functools import reduce


dfs = [df_pop, df_ethnic, df_income]
df_census = reduce(lambda left,right: pd.merge(left,right,on='Neighborhood'), dfs)
df_census.rename(columns={'After-Tax Household Income': 'Household Income'}, inplace=True)
df_census.head()

,Neighborhood,Total Population,Southeast Asian,Household Income
0,West Humber-Clairville,33312.0,490.0,59703.0
1,Mount Olive-Silverstone-Jamestown,32954.0,590.0,46986.0
2,Thistletown-Beaumond Heights,10360.0,120.0,57522.0
3,Rexdale-Kipling,10529.0,170.0,51194.0
4,Elms-Old Rexdale,9456.0,370.0,49425.0


### 2.2.3 Load Geodata

In [30]:
geospatial_df = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_df['PostalCode'] = geospatial_df['Postal Code']
geospatial_df.drop(['Postal Code'], axis=1, inplace=True)
geospatial_df.head()

,Latitude,Longitude,PostalCode
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


### 2.2.4 Merging all data into a single database

In [35]:
df = pd.merge(neighborhood_df, geospatial_df, on='PostalCode').drop(['PostalCode', 'Borough'], axis=1)
df = pd.merge(df, df_census, on='Neighborhood', how='left').dropna(how='any', axis=0).reset_index(drop=True)
df.head()

,Neighborhood,Latitude,Longitude,Total Population,Southeast Asian,Household Income
0,Victoria Village,43.725882,-79.315572,17510.0,95.0,43743.0
1,Regent Park,43.654260,-79.360636,10803.0,390.0,30794.0
2,Malvern,43.806686,-79.194353,43794.0,200.0,53425.0
3,Rouge,43.806686,-79.194353,46496.0,295.0,72784.0
4,Highland Creek,43.784535,-79.160497,12494.0,85.0,87321.0
